In [1]:
import os, sys, math
import argparse
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

sys.path.append(r'E:\TEST\ChangeFace')

In [2]:
from recognition.arcface.train import parse_args
from recognition.arcface.configs import config, default, update_config
from recognition.arcface.build_tensors import get_emb
from recognition.arcface.data_generator import DatasetMaker
from recognition.arcface.utils import init_vars, histplot_angles, print_variables

In [3]:
update_config(default.network, default.dataset, default.loss, default.initializer, default.optimizer)

In [4]:
class Metrics:
    def __init__(self):

        self.graph = tf.Graph()

        with self.graph.as_default():
            self.build()
            print_variables(tf.global_variables(), 'global_variables')

            conf = tf.ConfigProto()
            conf.allow_soft_placement = True
            self.sess = tf.Session(config=conf)

            # 读取保存变量
            restore_path = os.path.join(default.model_dir[0], f'v{default.restore_version[0]}')
            loader = tf.train.Saver()
            loader.restore(self.sess, restore_path)
            print(f'SUCCEED: restore from {restore_path}.')


    def build(self):
        dm = DatasetMaker()
        train_dataset = dm.read(default.dataset[0], cate='train', aug=False)
        train_dataset = train_dataset.shuffle(1000).batch(128)
        train_iterator = train_dataset.make_initializable_iterator()
        self.train_dataset_init = train_iterator.initializer
        train_imgs, self.train_labels = train_iterator.get_next()
        train_labels_onehot = tf.one_hot(self.train_labels, config.num_cls)
        print(f'SUCCEED: make train dataset from {default.dataset[0]}.')

        # 得到语义向量 - 已l2规范化
        self.emb, self.l2_cls_weight = get_emb(train_imgs, name='arcface', reuse=False, training=False,
                                               keep_prob=1, summary=False)

        self.predict, self.acc_num, self.train_angles = self._get_angles(self.emb, self.train_labels)
        self.angles1 = self.train_angles * train_labels_onehot
        self.angles2 = self.train_angles * (1. - train_labels_onehot)

        """用于测试的部分"""
        test_dataset = dm.read(default.dataset[0], cate='test', aug=False)
        test_dataset = test_dataset.shuffle(1000).batch(128)
        test_iterator = test_dataset.make_initializable_iterator()
        self.test_dataset_init = test_iterator.initializer
        test_imgs, self.test_labels = test_iterator.get_next()
        test_labels_onehot = tf.one_hot(self.test_labels, config.num_cls)
        print(f'SUCCEED: make test dataset from {default.dataset[0]}.')

        # 得到语义向量 - 已l2规范化
        emb, _ = get_emb(test_imgs, name='arcface', reuse=True, training=False, keep_prob=1, summary=False)

        self.test_predict, self.test_acc_num, test_angles = self._get_angles(emb, self.test_labels)
        self.test_angles1 = test_angles * test_labels_onehot
        self.test_angles2 = test_angles * (1. - test_labels_onehot)

    def _get_angles(self, emb, labels):
        cosines = tf.matmul(emb, self.l2_cls_weight)

        radians = tf.acos(cosines)
        angles = radians * 180 / math.pi

        predict = tf.argmax(cosines, axis=1, output_type=labels.dtype)
        acc_num = tf.reduce_sum(tf.cast(tf.equal(predict, labels), tf.float32))

        return predict, acc_num, angles    

    def close(self):
        self.sess.close()

    def check_dir(self, fdir):
        if not os.path.isdir(fdir):
            os.makedirs(fdir)

    def get_confusion_matrix(self, plot=False):

        self._get_confusion_matrix('train', plot)
        self._get_confusion_matrix('test', plot)

    def _get_confusion_matrix(self, cate='train', plot=True):
        com_num = 0
        cum_labels, cum_predict = [], []
        if cate == 'train':
            labels, predict = self.train_labels, self.predict
            ds_init = self.train_dataset_init
        else:
            labels, predict = self.test_labels, self.test_predict
            ds_init = self.test_dataset_init
        # 混淆矩阵 [预测类别，实际类别]=模型预测数目
        confusion_matrix = np.zeros(shape=[config.num_cls, config.num_cls])
        self.sess.run(ds_init)
        while True:
            try:
                _labels, _predict = self.sess.run([labels, predict])
                com_num += _labels.size
                cum_labels.extend(_labels)
                cum_predict.extend(_predict)
                print(f'\r{cate}: com_num: {com_num}...', end='')
            except:
                break

        lp = list(zip(cum_labels, cum_predict))
        rcn = np.array([[i[0], i[1], lp.count(i)] for i in set(zip(cum_labels, cum_predict))])
        confusion_matrix[rcn[:, 0], rcn[:, 1]] += rcn[:, 2]


        result_dir = f'../record/result/{default.restore_version[0]}'
        self.check_dir(result_dir)
        np.save(os.path.join(result_dir, f'{cate}_confusion_matrix'), confusion_matrix)

In [5]:
metrics = Metrics()



Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.
SUCCEED: make train dataset from f.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).


Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use keras.layers.dense instead.

SUCCEED: make test dataset from f.


****************************************************************************************************
global_variables:



----------------------------------------------------------------------------------------------------
arcface/darknet/conv2d/kernel:0		(3, 3, 3, 16)		432		0.00%
----------------------------------------------------------------------------------------------------
arcface/darknet/batch_normalization/gamma:0		(16,)		16		0.00%
----------------------------------------------------------------------------------------------------
arcface/darknet/batch_normalization/beta:0		(16,)		16		0.00%
----------------------------------------------------------------------------------------------------
arcface/darknet/unit0/conv2d/kernel:0		(3, 3, 16, 32)		4608		0.04%
----------------------------------------------------------------------------------------------------
arcface/darknet/unit0/block0/batch_normalization/gamma:0		(32,)		32		0.00%
----------------------------------------------------------------------------------------------------
arcface/darknet/unit0/block0/batch_normalization/beta:0		(32,)		32		0.

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ../record/mdoels/v01-99
SUCCEED: restore from ../record/mdoels/v01-99.


In [6]:
metrics.get_confusion_matrix()

train: com_num: 19040...

IndexError: index 427 is out of bounds for axis 0 with size 284

In [ ]:
metrics.close()